In [1]:
from torch.utils.data import Dataset
import torch
import numpy as np
import res.process_data.dire_and_coor as dc

In [2]:
# get self avoid polymer
self_avoid_directions = np.load('../../data/self_avoid/16/directions.npy')
self_avoid_coordinates = np.load('../../data/self_avoid/16/coorinates.npy')

In [3]:
# get random polymer
random = np.random.randint(0, 4, size=(120000, 14, 1)) / 4
a = np.zeros((120000, 15, 1))
# make the first one is forward
for i in range(len(random)):
    a[i] = np.vstack(([0], random[i]))

In [4]:
# remove self avoid data
random_coordinate = dc.direction_coordinate_three(a, 16)

In [5]:
def check_fold_cross(coordinate):
    """
    check if a single polymer is folding or crossing
    """
    assert np.shape(coordinate) == (16, 2)
    folding_count = 0
    crossing_count = 0
    for i in range(0, len(coordinate)):
        temp = coordinate[i]
        if (temp == coordinate[:i]).all(axis=1).any():
            if np.array_equal(temp, coordinate[:i][-2]):
                folding_count += 1
            else:
                crossing_count += 1
    return [folding_count, crossing_count]

In [17]:
c = []
for i in range(len(random_coordinate)):
    if check_fold_cross(random_coordinate[i]) != [0,0]:
        c.append(i) # self avoid

In [18]:
output = np.zeros((len(c), 14, 1))
for i in c:
    output[i] = random[i]

IndexError: index 119292 is out of bounds for axis 0 with size 119292

In [ ]:
class tensor_dataset(Dataset):
    def __init__(self, input_tensor, row_dim, column_dim):
        self.input_tensor = input_tensor
        self.row_dim = row_dim
        self.column_dim = column_dim

    def __len__(self):
        return len(self.input_tensor)

    def __getitem__(self, index: int):
        tensor_id = index
        tensor = self.input_tensor[tensor_id]
        # convert the tensor from [x,y] to [1, x, y]
        tensor = torch.reshape(tensor, [1, self.row_dim, self.column_dim])
        # all label is 1(no label)
        tensor_label = torch.tensor(1)
        return tensor, tensor_label